In [10]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate


# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
    memory_key="chat_history",
    return_messages=True
)

examples = [
    {
        "question": "Describe 'Home Alone' with three emojis that represent the movie",
        "answer": "🏠🎄💪",
    },
    {
        "question": "Describe 'Harry Potter' with three emojis that represent the movie",
        "answer": "🪄📚🧑🏼‍🎓",
    },
    {
        "question": "Describe 'Hunger Game' with three emojis that represent the movie",
        "answer": "🏹🕹️🐦‍⬛",
    }
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], 
    template="This is answer: {answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Describe 'Harry Potter'"))

This is answer: 🏠🎄💪

This is answer: 🪄📚🧑🏼‍🎓

This is answer: 🏹🕹️🐦‍⬛

Question: Describe 'Harry Potter'
